In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import bisect
import datetime
import enum
import json
import math
import pathlib

In [ ]:
import fumbblapi
import srdata
import srpages
import srpoints
import srschedule
import srtime
import srtournament

In [ ]:
sr_data_path = pathlib.Path('../sr-data')
sr_pages_path = pathlib.Path('../sr-pages')

In [ ]:
TI = srdata.TournamentIdx
srdata.load(sr_data_path)

In [ ]:
updated = srtime.now().strftime('%Y-%m-%d %H:%M')
updated

## Add New Tournaments

In [ ]:
tournament_format_to_elim = {'King': 'E', 'Knockout': 'E', 'RoundRobin': 'N', 'Swiss': 'N'}
titles = {row[TI.TITLE] for row in srdata.data["tournaments"] if TI.TITLE < len(row) and row[TI.TITLE]}
print('Current titles: ' + ', '.join(sorted(titles)))
slots = set(srdata.data["slot"].keys())
print('Current slots: ' + ', '.join(sorted(slots)))

In [ ]:
class ClassIdx(enum.IntEnum):
  FORMAT = 0
  RANK = 1
  LEVEL = 2
  TEAMS = 3
CI = ClassIdx

In [ ]:
new_rows = []
for groupId in srdata.data["groups"]:
    group_tournaments = fumbblapi.get__group_tournaments(groupId)
    for tournament in group_tournaments:
        tournamentId = tournament["id"]
        class_parts = [None] * (CI.LEVEL + 1)
        if tournamentId not in srdata.data["tournament"]:
            print(f'{tournament["name"].strip()} [{tournamentId}|{groupId}]')
            print('-' * 100)
            # determining type
            old_group_tournaments = fumbblapi.old_get__group_tournaments(groupId)
            tournament_format = srtournament.format_(old_group_tournaments, tournamentId)
            class_parts[CI.FORMAT] = tournament_format_to_elim[tournament_format]
            # asking for tournament name
            name = input('Name? (Hit Enter to keep current one) ').strip()
            if not name:
                name = tournament["name"].strip()
            # asking for main class and level
            while class_parts[CI.RANK] not in {'MI', 'MA', 'QU'}:
                class_parts[CI.RANK] = input('Main Class? (MI/MA/QU; Enter for MI) ').strip().upper()
                if not class_parts[CI.RANK]:
                    class_parts[CI.RANK] = 'MI'
            while True:
                class_parts[CI.LEVEL] = input('Level? (integer; Enter for 1) ').strip()
                if not class_parts[CI.LEVEL]:
                    class_parts[CI.LEVEL] = '1'
                if not class_parts[CI.LEVEL].isdecimal():
                    continue
                class_ = '/'.join(class_parts)
                points_keys = {k for k in srdata.data["points"].keys() if k.startswith(class_)}
                if points_keys:
                    break
            if class_parts[CI.RANK] != 'QU':
                # asking for title
                while True:
                    title = input('Title? (Enter for none) ').strip().upper()
                    if title and title not in titles:
                        sure = None
                        while sure not in ('Y', 'N'):
                            sure = input('Are you sure to create a new title? (Y/N) ').strip().upper()
                        if sure == 'N':
                            continue
                    break
                # asking for slot
                slot = None
                while slot not in slots:
                    slot = input('Slot? (Enter for R) ').strip().upper()
                    if not slot:
                        slot = 'R'
            if class_parts[CI.RANK] == 'MA':
                # asking whether to join unchained qualifiers
                qurows = [r for r in srdata.data["tournament"] if r[TI.TOP_ID] is None and r[TI.CLASS].startswith('QU')]
                if qurows:
                    print('Check whether the following tournaments are qualifiers of this one (Y/N): ')
                for row2 in qurows:
                    yn = None
                    while yn not in ('Y', 'N'):
                        yn = input(f' {row2[TI.NAME]} [{row2[TI.ID]}|{row2[TI.GROUP_ID]}]').srtip().upper()
                    if yn == 'Y':
                        row2[TOP_TOURNAMENT] = tournamentId

            # determining teams
            if class_parts[CI.FORMAT] == 'E':
                schedule = fumbblapi.get__tournament_schedule(tournamentId)
                teams = len(srschedule.teams(schedule))
                class_parts.append(str(teams))
                class_ += f'/{teams}'
            # we now have the tournament class
            top_id = (None if class_parts[CI.RANK] == 'QU' else 0)
            winner_id = tournament.get("winner", {}).get("id")
            row = [None] * (TI.WINNER_ID + 1)
            row[TI.ID] = tournamentId
            row[TI.TOP_ID] = top_id
            row[TI.GROUP_ID] = groupId
            row[TI.NAME] = name
            row[TI.CLASS] = class_
            row[TI.WINNER_ID] = winner_id
            if class_parts[CI.RANK] != 'QU':
                row.extend([None] * (TI.EXIT_WEEKNR - TI.WINNER_ID))
                row[TI.TITLE] = title
                row[TI.FIRST_SLOT_GROUP] = slot
            print(row)
            print()
            # add row to the tournament data
            new_rows.append(row)
            srdata.data["tournament"][row[TOURNAMENT]] = row
            bisect.insort(srdata.data["tournaments"], row)


## Refresh 'In Progress' Tournaments

In [ ]:
for row in srdata.data["tournaments"]:
    if row[TI.TOP_ID] or len(row) < TI.ENTER_WEEKNR or row[TI.ENTER_WEEKNR]:
        continue
    group_tournaments = fumbblapi.get__group_tournaments(row[TI.GROUP_ID])
    tournament = [t for t in group_tournaments if t["id"] == row[TI.ID]][0]
    if tournament["status"] != 'Completed':
        continue
    schedule = fumbblapi.get__tournament_schedule(row[TI.ID])
    report_weeknr = row[TI.ENTER_WEEKNR] = srschedule.report_weeknr(schedule)
    row[TI.EXIT_WEEKNR] = report_weeknr + (78 if row[-4] else 52)
    print('Completed: ', row)
    prev, next_ = srtournament.prevnext_title(row)
    if prev and row[TI.ENTER_WEEKNR] < prev[TI.EXIT_WEEKNR]:
        print(f'Exit weeknr changed for the following tournament from {prev[TI.EXIT_WEEKNR]} to {row[TI.ENTER_WEEKNR]}:')
        prev[TI.EXIT_WEEKNR] = row[TI.ENTER_WEEKNR]
        print(prev)

## Save TOURNAMENTS.JSON

In [ ]:
t = '[\n' + ',\n'.join(json.dumps(row) for row in srdata.data["tournaments"]) + '\n]'

In [ ]:
with (sr_data_path / 'tournaments.json').open('w') as f:
    f.write(t)

## Tournaments Table

In [ ]:
curr_weeknr = srtime.current_weeknr()
weeknr_report_number = {w: r for r, w in enumerate(srtime.report_weeknrs(), 1)}

In [ ]:
nameurlfs = 'https://fumbbl.com/p/group?op=view&at=1&group={group}&p=tournaments&show={tournament}&showallrounds=1'

In [ ]:
rank_order = ('MA', 'QU', 'MI')
pending_rows_ = []
finished_rows_ = []
for row in srdata.data["tournaments"]:
    name = row[TI.NAME]
    nameurl = nameurlfs.format(group=row[TI.GROUP_ID], tournament=row[TI.ID])
    tname = f'[{name}|{nameurl}]'
    class_ = row[TI.CLASS]
    class_parts = class_.split('/')
    if len(class_parts) == CI.LEVEL + 1:
        format__, rank, level = class_parts
        teams = ''
    elif len(class_parts) == CI.TEAMS + 1:
        format__, rank, level, teams = class_parts
    keyclass = srpoints.get_keyclass(row[TI.CLASS])
    points = srdata.data['points'][keyclass]
    first_slot_group = ' '
    enter_date, exit_date = ' ', ' '
    enter_report, exit_report = None, None
    if row[TI.TOP_ID] is None:
        top_name = ''
        top_enter_weeknr = 0
        top_exit_weeknr = 0
    elif row[TI.TOP_ID] == 0:
        first_slot_group = row[TI.FIRST_SLOT_GROUP]
        top_name = name
        top_enter_weeknr = row[TI.ENTER_WEEKNR] or 0
        top_exit_weeknr = row[TI.EXIT_WEEKNR] or 0
        if top_enter_weeknr:
            enter_date = srtime.weeknr_firstdate(top_enter_weeknr).isoformat()
            exit_date = srtime.weeknr_firstdate(row[TI.EXIT_WEEKNR]).isoformat()
            enter_report = weeknr_report_number[top_enter_weeknr]
            exit_report = weeknr_report_number.get(row[TI.EXIT_WEEKNR])
    else:
        top_row = srdata.data["tournament"][row[TI.TOP_ID]]
        top_name = top_row[TI.NAME]
        top_enter_weeknr = top_row[TI.ENTER_WEEKNR] or 0
        top_exit_weeknr = top_row[TI.EXIT_WEEKNR] or 0
    if enter_report:
        tenter_date = f'[{enter_date}|SR-Report-{enter_report}]'
    else:
        tenter_date = enter_date
    if exit_report:
        texit_date = f'[{exit_date}|SR-Report-{exit_report}]'
    elif (
            row[TI.TOP_ID] == 0
            and row[TI.ENTER_WEEKNR]
            and row[TI.EXIT_WEEKNR]
            and row[TI.ENTER_WEEKNR] + 52 < row[TI.EXIT_WEEKNR]
        ):
        texit_date = f'({exit_date})'
    else:
        texit_date = exit_date
    if top_enter_weeknr:# and top_enter_weeknr <= curr_weeknr:
        sort_values = [-top_enter_weeknr, (rank == 'MI'), top_name, -int(level), row[TI.ID], top_exit_weeknr]
        rows = finished_rows_
    else:
        sort_values = [-top_enter_weeknr, (rank == 'MI'), top_name, -int(level), row[TI.ID]]
        rows = pending_rows_
    table_values = [tname, rank, level, format__, teams, points, first_slot_group, tenter_date, texit_date]
    if first_slot_group in ('NE'):
        continue
    bisect.insort(rows, [sort_values, table_values])
pending_rows = [t[1] for t in pending_rows_]
finished_rows = [t[1] for t in finished_rows_]

### Pending Tournaments

The cell below generates the data of pending tournaments. After run, go below to the _Generate content_ section.

In [ ]:
pagename = f'SR-PendingTournaments'
title = f'OBC Sport SR Rankings Pending Tournaments'
tournaments_rows = [row[:-2] for row in pending_rows]
tournaments = srpages.table(tournaments_rows,
        align='LCCCCLC', 
        header=['Name', 'Rank', 'Level', 'Format', 'Teams', 'Points', 'First Slot Group'],
)

### Tournaments of Year

#### Set the Year

The cells below provide varios ways to set the year for the generated page. 

The first option is to edit it manually. Edit the next cell and go to _Filter Tournaments by Year_ subsection:

In [ ]:
yyyy = 2018

The following cell creates a generator which yields all the years involved, one by one. Normally the last two years should get refreshed for every Reports which is done by the second next cell. Run the first only if you want to refresh tournament pages for all years.

In [ ]:
years = sorted(set(
        srtime.weeknr_firstdate(row[TI.ENTER_WEEKNR]).year
        for row in srdata.data["tournaments"]
        if TI.ENTER_WEEKNR < len(row)
        and row[TI.ENTER_WEEKNR]
))
gen = iter(years)
print('; '.join(str(y) for y in years))

In [ ]:
years = years[-2:]
gen = iter(years)
print('; '.join(str(y) for y in years))

##### Year Selection Loop

You should run the cell below and go forward to the _Filter Tournaments by Year_ subsection for every year in the collection.

In [ ]:
yyyy = next(gen)
print(yyyy)

#### Filter Tournaments by Year

The cell below generates the tournaments data of the actual year. After run, go below to the Generate content section.

In [ ]:
pagename = f'SR-TournamentsOf{yyyy}'
title = f'OBC Sport SR Rankings Tournaments of {yyyy}'
tournaments_rows = [t[1] for t in finished_rows_ if srtime.weeknr_firstdate(-t[0][0]).year == yyyy]
tournaments = srpages.table(tournaments_rows,
        align='LCCCCLCCC', 
        header=['Name', 'Rnk', 'Lvl', 'Fmt', 'Tms', 'Points', 'FSG', 'Enter Date', 'Exit Date'],
)

### Generate content

In [ ]:
template_filename = 'SR-Tournaments.template.phpwiki'
with (sr_pages_path / template_filename).open() as f:
    content = f.read().format(updated=updated, title=title, tournaments=tournaments)

### Edit Manually

Without the closed source `fumbbl_session` library, one should manually edit the page generated by the cell below. Please copy the result of the second next cell below to the clipboard and paste it to the edit box of the Tournaments page.

In [ ]:
print(f'https://fumbbl.com/help:{pagename}?action=edit')

In [ ]:
print(content)

### Edit Automatically

In [ ]:
import fumbbl_session as S

In [ ]:
login_data_path = pathlib.Path('login.json')

In [ ]:
with login_data_path.open() as f:
    _login = json.load(f)
S.log_in(**_login)

In [ ]:
S.helppage.edit(pagename, content)

### Verify Result

In [ ]:
print(f'https://fumbbl.com/help:{pagename}')